<a href="https://colab.research.google.com/github/AnhVietPham/Deep-Learning/blob/main/Deep-Learning/DL-Pytorch/dataset-normalization/Datasets_normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from torch.utils.tensorboard import SummaryWriter

from torch.utils.data import DataLoader

import time
import pandas as pd
import json
from IPython.display import clear_output

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)     # On by default, leave it here for clarity

In [24]:
train_set = torchvision.datasets.FashionMNIST(
    root = './data',
    train = True,
    download = True,
    transform = transforms.Compose([transforms.ToTensor()]))

# **Calculating mean And std The Easy Way**

In [25]:
print(len(train_set))
loader = DataLoader(train_set, batch_size=len(train_set), num_workers=1)
data = next(iter(loader))
print(len(data))
print(data[0].shape)
print(data[0].sum())
print(data[1])
print(len(data[1]))
data[0].mean(), data[0].std()

60000
2
torch.Size([60000, 1, 28, 28])
tensor(13455350.)
tensor([9, 0, 0,  ..., 3, 0, 5])
60000


(tensor(0.2860), tensor(0.3530))

# **Calculating mean And std The Hard Way**

In [26]:
loader = DataLoader(train_set, batch_size=1000, num_workers=1)
num_of_pixels = len(train_set) * 28 * 28
print(num_of_pixels)

47040000


In [27]:
total_sum = 0
for batch in loader:
  print(batch[0].sum())
  total_sum += batch[0].sum()

tensor(221796.0938)
tensor(223419.1562)
tensor(223897.0312)
tensor(226196.3906)
tensor(226385.4062)
tensor(222110.9688)
tensor(220036.5000)
tensor(224462.5312)
tensor(227501.3750)
tensor(228856.5312)
tensor(222647.2344)
tensor(224786.3125)
tensor(231266.4375)
tensor(220914.9219)
tensor(227135.8750)
tensor(216532.7500)
tensor(222205.8906)
tensor(220187.1719)
tensor(224376.4375)
tensor(226165.2812)
tensor(220904.8438)
tensor(227582.1875)
tensor(223741.4375)
tensor(220569.9688)
tensor(225460.0625)
tensor(226125.3594)
tensor(223911.4062)
tensor(221863.9688)
tensor(223762.7812)
tensor(224458.9844)
tensor(220922.9219)
tensor(225264.8594)
tensor(221296.6406)
tensor(223807.2188)
tensor(228280.7344)
tensor(224910.3750)
tensor(226677.9688)
tensor(221769.6250)
tensor(223601.9219)
tensor(218705.6875)
tensor(223314.9062)
tensor(223065.6094)
tensor(223229.0312)
tensor(228174.2812)
tensor(227195.8281)
tensor(221107.1250)
tensor(218621.8281)
tensor(224728.7812)
tensor(223405.7812)
tensor(224215.1562)


In [28]:
mean = total_sum/num_of_pixels
print(mean)

tensor(0.2860)


In [29]:
sum_of_squared_error = 0
for batch in loader:
  sum_of_squared_error += ((batch[0]-mean).pow(2)).sum()
std = torch.sqrt(sum_of_squared_error / num_of_pixels)

In [30]:
mean, std

(tensor(0.2860), tensor(0.3530))

# **Using The mean And std Values**

In [31]:
train_set_normal = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
          transforms.ToTensor()
        , transforms.Normalize(mean, std)
    ])
)

In [32]:
loader = DataLoader(
    train_set_normal,
    batch_size = len(train_set),
    num_workers = 1
)

data = next(iter(loader))
data[0].mean(), data[0].std()

(tensor(-1.8468e-07), tensor(1.))

# **Training With Normalized Data**

In [33]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
       
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        
        return t

In [38]:
# import modules to build RunBuilder and RunManager helper classes
from collections  import OrderedDict
from collections import namedtuple
from itertools import product

class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs

In [39]:
class RunManager():
    def __init__(self):
        
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None
        
        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None
        
        self.network = None
        self.loader = None
        self.tb = None
        
    def begin_run(self, run, network, loader):
        
        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1
        
        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment=f'-{run}')
        
        images, labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)

        self.tb.add_image('images', grid)
        self.tb.add_graph(
             self.network
            ,images.to(getattr(run, 'device', 'cpu'))
        )
        
    def end_run(self):
        self.tb.close()
        self.epoch_count = 0   

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0

    def end_epoch(self):
        
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time
        
        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy = self.epoch_num_correct / len(self.loader.dataset)
                
        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        
        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)
            self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)
        
        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results['loss'] = loss
        results["accuracy"] = accuracy
        results['epoch duration'] = epoch_duration
        results['run duration'] = run_duration
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')
        
        clear_output(wait=True)
        display(df)
        
    def track_loss(self, loss, batch):
        self.epoch_loss += loss.item() * batch[0].shape[0]
        
    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)
    
    def _get_num_correct(self, preds, labels):
        return preds.argmax(dim=1).eq(labels).sum().item()
    
    def save(self, fileName):
        
        pd.DataFrame.from_dict(
            self.run_data
            ,orient='columns'
        ).to_csv(f'{fileName}.csv')
        
        with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)

In [40]:
trainsets = {
    'not_normal': train_set,
    'normal': train_set_normal
}

In [42]:
params = OrderedDict(
    lr = [.01],
    batch_size = [1000],
    num_workers = [1],
    device = ['cuda'],
    trainset = ['not_normal', 'normal']
)

m = RunManager()
for run in RunBuilder.get_runs(params):
  device = torch.device(run.device)
  network = Network().to(device)
  loader = DataLoader(
      trainsets[run.trainset],
      batch_size = run.batch_size,
      num_workers = run.num_workers
  )
  optimizer = optim.Adam(network.parameters(), lr = run.lr)

  m.begin_run(run, network, loader)
  for epoch in range(20):
    m.begin_epoch()
    for batch in loader:
      images = batch[0].to(device)
      labels = batch[1].to(device)
      preds = network(images)
      loss = F.cross_entropy(preds, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      m.track_loss(loss, batch)
      m.track_num_correct(preds, labels)
    m.end_epoch()
  m.end_run()
m.save('results')

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers,device,trainset
0,1,1,1.102377,0.582600,4.618623,5.360122,0.01,1000,1,cuda,not_normal
1,1,2,0.608606,0.766333,4.628802,10.138204,0.01,1000,1,cuda,not_normal
2,1,3,0.499433,0.814350,4.597122,14.858428,0.01,1000,1,cuda,not_normal
3,1,4,0.444292,0.838133,4.740224,19.727041,0.01,1000,1,cuda,not_normal
4,1,5,0.403925,0.851717,4.664985,24.535421,0.01,1000,1,cuda,not_normal
5,1,6,0.376696,0.860750,4.639808,29.299633,0.01,1000,1,cuda,not_normal
6,1,7,0.360233,0.866900,4.599237,34.036838,0.01,1000,1,cuda,not_normal
7,1,8,0.343036,0.872583,4.609787,38.796014,0.01,1000,1,cuda,not_normal
8,1,9,0.332528,0.877100,4.626369,43.566612,0.01,1000,1,cuda,not_normal
9,1,10,0.324560,0.879233,4.614933,48.310462,0.01,1000,1,cuda,not_normal
